In [1]:
import sys
import os
import sklearn
import re

import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import pandas_profiling as pdpf

import seaborn as sns 

import tsfresh
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh import extract_features
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction.settings import from_columns
from tsfresh.utilities.dataframe_functions import impute_dataframe_range

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

mpl.rc('axes', labelsize=10)
mpl.rc('xtick', labelsize=8)
mpl.rc('ytick', labelsize=8)

In [2]:
FILE_PATH = "/kaggle/input/wids-23"
def load_data(filename, path = FILE_PATH):
    return pd.read_csv(os.path.join(path, filename))
train_data = load_data(filename = "train_data.csv")

In [3]:
def reduce_mem_usage(dataframe, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_memory = dataframe.memory_usage().sum() / 1024**2
    for col in dataframe.columns:
        col_type = dataframe[col].dtypes
        if col_type in numerics:
            c_min = dataframe[col].min()
            c_max = dataframe[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    dataframe[col] = dataframe[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    dataframe[col] = dataframe[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    dataframe[col] = dataframe[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    dataframe[col] = dataframe[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    dataframe[col] = dataframe[col].astype(np.float32)
                else:
                    dataframe[col] = dataframe[col].astype(np.float64)
    end_memory = dataframe.memory_usage().sum() / 1024**2
    print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_memory, 100 * (start_memory - end_memory) / start_memory)) if verbose else print('Reduced to {:5.2f}'.format(end_memory))
    return dataframe

In [4]:
train_data = reduce_mem_usage(train_data)

Mem. usage decreased to 352.24 Mb (50.1% reduction)


In [5]:
# fc_params = ['contest-pevpr-sfc-gauss-14d@pevpr__minimum', 
#              'contest-slp-14d@slp__standard_deviation', 
#              'contest-wind-h100-14d@wind-hgt-100__maximum', 
#              'contest-wind-h850-14d@wind-hgt-850__absolute_maximum',
#              'contest-wind-uwnd-9@5-14d@wind-uwnd-9@5__sum_values',
#              'contest-precip-14d@precip__minimum', 'nasa0__absolute_maximum',
#              'contest-wind-uwnd-@50-14d@wind-uwnd-@50__variance',
#              'contest-rhum-sig995-14d@rhum__variance', 
#              'contest-wind-h850-14d@wind-hgt-850__standard_deviation',
#              'contest-pevpr-sfc-gauss-14d@pevpr__absolute_maximum', 
#              'contest-wind-vwnd-@50-14d@wind-vwnd-@50__absolute_maximum', 
#              'nmme-prate-56w@gfdl__minimum', 'contest-rhum-sig995-14d@rhum__maximum', 
#              'contest-slp-14d@slp__median', 'contest-wind-vwnd-9@5-14d@wind-vwnd-9@5__mean', 
#              'nmme-prate-56w@ccsm3__minimum', 'nmme-prate-34w@ccsm3__minimum', 'nmme0-prate-34w@ccsm30__minimum',
#              'elevation@elevation__absolute_maximum', 
#              'contest-wind-vwnd-9@5-14d@wind-vwnd-9@5__maximum', 
#              'contest-wind-uwnd-9@5-14d@wind-uwnd-9@5__root_mean_square', 
#              'nmme0-prate-56w@cancm30__minimum', 
#              'contest-precip-14d@precip__standard_deviation', 
#              'ccsm30__variance', 'nmme-prate-34w@ccsm3__variance', 
#              'nmme0-prate-34w@cfsv@0__minimum', 'nmme0-prate-34w@cancm40__minimum', 
#              'contest-wind-uwnd-@50-14d@wind-uwnd-@50__root_mean_square', 
#              'contest-wind-vwnd-9@5-14d@wind-vwnd-9@5__minimum', 
#              'nmme-prate-56w@gfdl__median', 'nmme0-prate-34w@ccsm40__minimum', 
#              'nmme0-prate-34w@gfdl0__standard_deviation', 'nmme-prate-34w@nasa__median', 
#              'nmme-prate-34w@gfdlflora__standard_deviation']

In [6]:
col_na = [
    'nmme0-tmp2m-34w__ccsm30', 
    'nmme-tmp2m-56w__ccsm3', 
    'nmme-prate-34w__ccsm3', 
    'nmme0-prate-56w__ccsm30', 
    'nmme0-prate-34w__ccsm30', 
    'nmme-prate-56w__ccsm3', 
    'nmme-tmp2m-34w__ccsm3']

g_means =  ['nmme0-tmp2m-34w__nmme0mean', 
 'nmme-tmp2m-56w__nmmemean', 
 'nmme-prate-34w__nmmemean', 
 'nmme0-prate-56w__nmme0mean', 
 'nmme0-prate-34w__nmme0mean', 
 'nmme-prate-56w__nmmemean', 
 'nmme-tmp2m-34w__nmmemean']


g_1 = ['nmme0-tmp2m-34w__cancm30',
'nmme0-tmp2m-34w__cancm40',
'nmme0-tmp2m-34w__ccsm40',
'nmme0-tmp2m-34w__cfsv20',
'nmme0-tmp2m-34w__gfdlflora0',
'nmme0-tmp2m-34w__gfdlflorb0',
'nmme0-tmp2m-34w__gfdl0',
'nmme0-tmp2m-34w__nasa0']

g_2 = ['nmme-tmp2m-56w__cancm3',
'nmme-tmp2m-56w__cancm4',
'nmme-tmp2m-56w__ccsm4',
'nmme-tmp2m-56w__cfsv2',
'nmme-tmp2m-56w__gfdl',
'nmme-tmp2m-56w__gfdlflora',
'nmme-tmp2m-56w__gfdlflorb',
'nmme-tmp2m-56w__nasa']

g_3 = ['nmme-prate-34w__cancm3',
'nmme-prate-34w__cancm4',
'nmme-prate-34w__ccsm4',
'nmme-prate-34w__cfsv2',
'nmme-prate-34w__gfdl',
'nmme-prate-34w__gfdlflora',
'nmme-prate-34w__gfdlflorb',
'nmme-prate-34w__nasa']

g_4 = [ 'nmme0-prate-56w__cancm30',
'nmme0-prate-56w__cancm40',
'nmme0-prate-56w__ccsm40',
'nmme0-prate-56w__cfsv20',
'nmme0-prate-56w__gfdlflora0',
'nmme0-prate-56w__gfdlflorb0',
'nmme0-prate-56w__gfdl0',
'nmme0-prate-56w__nasa0']

g_5 = ['nmme0-prate-34w__cancm30',
'nmme0-prate-34w__cancm40',
'nmme0-prate-34w__ccsm40',
'nmme0-prate-34w__cfsv20',
'nmme0-prate-34w__gfdlflora0',
'nmme0-prate-34w__gfdlflorb0',
'nmme0-prate-34w__gfdl0',
'nmme0-prate-34w__nasa0']

g_6 = ['nmme-prate-56w__cancm3',
'nmme-prate-56w__cancm4',
'nmme-prate-56w__ccsm4',
'nmme-prate-56w__cfsv2',
'nmme-prate-56w__gfdl',
'nmme-prate-56w__gfdlflora',
'nmme-prate-56w__gfdlflorb',
'nmme-prate-56w__nasa']

g_7 = ['nmme-tmp2m-34w__cancm3',
'nmme-tmp2m-34w__cancm4',
'nmme-tmp2m-34w__ccsm4',
'nmme-tmp2m-34w__cfsv2',
'nmme-tmp2m-34w__gfdl',
'nmme-tmp2m-34w__gfdlflora',
'nmme-tmp2m-34w__gfdlflorb',
'nmme-tmp2m-34w__nasa']


In [7]:
nmme_tm2m_34w_features = train_data.columns[train_data.columns.str.startswith('nmme0-tmp2m-34w')].tolist()
nmme_tm2m_56w_features = train_data.columns[train_data.columns.str.startswith('nmme-tmp2m-56w')].tolist()
c14 = [c for c in train_data.columns if "14" in c][:14]
good_feats = nmme_tm2m_34w_features+nmme_tm2m_56w_features+c14

In [8]:
target = 'contest-tmp2m-14d__tmp2m'

def scaling(train, test):
    std_scaler = StandardScaler()
    not_to_touch = ['index',
                    'id',
                    'startdate',
                    'climateregions__climateregion',
                    'lat',
                    'lon',
                    'contest-tmp2m-14d__tmp2m']
    num_data = [f for f in train.select_dtypes(include= [np.number]).columns if f not in not_to_touch]
    train[num_data] = std_scaler.fit_transform(train[num_data])
    test[num_data] = std_scaler.transform(test[num_data])
    return train, test

def create_id(train, test):
    df = pd.concat([train, test], axis= 0)
    df['id'] = df.groupby(['lat', 'lon'], axis= 0).ngroup()
    df = df.drop(['lat', 'lon'], axis= 1)
    train = df.iloc[:len(train)]
    test = df.iloc[len(train):].drop(target, axis= 1)
    return train, test


def fill_na(data):
    df = data
    missing_columns = df.columns[df.isnull().any()].to_list()
    for col in missing_columns:
        df[col].ffill()
    return df

def fill_na_ver(data):
    df = data
    gs = [g_1, g_2, g_3, g_4, g_5, g_6, g_7]
    zip_cols = zip(col_na, gs, g_means)
    for c, g, m in zip_cols:
        df[c] = (df[m]*9) - df[g].sum(1)
        df = df.ffill()
    return df

def high_corr(data, threshold= 0.8):
    not_to_touch = ['index',
                    'id',
                    'startdate',
                    'climateregions__climateregion',
                    'lat',
                    'lon',
                    'contest-tmp2m-14d__tmp2m']
    df = data
    corr_matrix = df.corr().abs()
    mask = np.triu(np.ones_like(corr_matrix, dtype= bool))
    reduced_matrix = corr_matrix.mask(mask)
    feats_to_drop = [c for c in reduced_matrix.columns if any(reduced_matrix[c] > threshold)]
    feats_to_drop = [f for f in feats_to_drop if f not in not_to_touch]
    return feats_to_drop


def cat_encoding(train, test):
    cat_encoder = LabelEncoder()
    train['climateregions__climateregion'] = cat_encoder.fit_transform(train['climateregions__climateregion'])
    test['climateregions__climateregion'] = cat_encoder.transform(test['climateregions__climateregion'])
    return train, test

def process_date(train, test):
    df = pd.concat([train, test], axis= 0)
    df['day'] = pd.DatetimeIndex(df['startdate']).day
    df['month'] = pd.DatetimeIndex(df['startdate']).month
    df['quarter'] = pd.DatetimeIndex(df['startdate']).quarter
    df = df.drop('startdate', axis= 1)
    train = df.iloc[:len(train)]
    test = df.iloc[len(train):].drop(target, axis= 1)
    return train, test
    
def preprocess(train, test):
    train, test = create_id(train, test)
    train, test = process_date(train, test)
    train = fill_na_ver(train)
    feats_to_drop = high_corr(train)
    train, test = scaling(train, test)
    train, test = cat_encoding(train, test)
    y_train = train[target]
    x_train = train.drop(target, axis= 1)
    return x_train, y_train, test, feats_to_drop

In [9]:
test_data = load_data("test_data.csv")
test_data = reduce_mem_usage(test_data)

Mem. usage decreased to 29.48 Mb (49.7% reduction)


In [10]:
x_train, y_train, test, feats_to_drop = preprocess(train_data, test_data)

In [11]:
features = [c for c in x_train.columns if c not in feats_to_drop]

In [12]:
c14 = [c for c in x_train.columns if "14" in c][:14]+nmme_tm2m_34w_features+nmme_tm2m_56w_features

In [13]:
import optuna
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from catboost import Pool, cv, CatBoostRegressor
from xgboost import XGBRegressor
from sklearn import linear_model
import warnings
warnings.filterwarnings('ignore')

### Lasso

In [14]:
rr = []
rr1 = []
rr2 = []

for j in range(x_train["id"].nunique()):

    y1 = y_train[x_train["id"]==j] 
    x1 = x_train.loc[x_train["id"]==j][c14]
    test1 = test.loc[test["id"]==j][c14]
    
    p = -28
    
    x1_conc = pd.concat([x1, test1])
    x1_conc[nmme_tm2m_34w_features] = x1_conc[nmme_tm2m_34w_features].shift(-14).ffill()
    x1_conc[nmme_tm2m_56w_features] = x1_conc[nmme_tm2m_56w_features].shift(p).ffill(limit=20)
    x2 = x1_conc.iloc[:-61,:]
    test2 = x1_conc.iloc[-61:p+20,:]
    

    
    index = x_train.loc[x_train["id"]==j]["index"]
    index1 = test.loc[test["id"]==j]["index"]
    index2 = index1[:p+20]

    
    clf = linear_model.Lasso(alpha=0.019, max_iter=10000)
    
    clf.fit(x1, y1)
    train_res= clf.predict(x1)
    test_res = clf.predict(test1)
    
    clf.fit(x2, y1)
    train_res_1= clf.predict(x2)
    test_res_1 = clf.predict(test2)
    
    
    


    df_test   = pd.DataFrame(data=test_res,  columns = ["contest-tmp2m-14d__tmp2m"], index=index1)
    df_test_1   = pd.DataFrame(data=test_res_1,  columns = ["contest-tmp2m-14d__tmp2m"], index=index2)
    
    y_copy = y_train.copy()
    y_copy[x_train["id"]==j] = y_copy[x_train["id"]==j] - train_res
    
    rr.append(df_test)
    rr1.append(df_test_1)

In [15]:
sub = pd.concat(rr).reset_index()
sub1 = pd.concat(rr1).reset_index()
sub.loc[sub["index"].isin(sub1["index"].values),'contest-tmp2m-14d__tmp2m'] = sub1['contest-tmp2m-14d__tmp2m'].values

In [16]:
def split_train_eval(x, y, test_ratio):
    test_set_size = int(len(x) * (1 - test_ratio))
    return x.iloc[:test_set_size], y.iloc[:test_set_size], x.iloc[test_set_size:], y.iloc[test_set_size:]

In [17]:
x_train_final = x_train[features]
x_train_set, y_train_set, x_eval_set, y_eval_set = split_train_eval(x_train_final, y_train, 0.3)

### LGBM with optuna

In [18]:
def objective(trial):
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'objective': 'regression',
#               'max_depth': -1,
        'learning_rate': 0.1,
        "boosting": "gbdt",
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0),
        "bagging_freq": 5,
        "bagging_fraction": trial.suggest_float('bagging_fraction', 0.1, 1.0),
        "feature_fraction": trial.suggest_float('feature_fraction', 0.4, 1.0),
        "metric": 'rmse',
        "verbosity": 0,
        "num_boost_round": 1000,
        "early_stopping_rounds": 10
    }
    d_train = lgb.Dataset(x_train_set, label= y_train_set)
    d_valid = lgb.Dataset(x_eval_set, label= y_eval_set)
    watchlist = [d_valid]
    pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'rmse')
    model = lgb.train(params,
                      train_set=d_train,
                      valid_sets=watchlist,
                      callbacks=[pruning_callback])

    y_pred = model.predict(x_eval_set, num_iteration=model.best_iteration)
    score = mean_squared_error(y_eval_set, y_pred, squared= False)
    return score

In [19]:
study_lgbm = optuna.create_study(direction= 'minimize', study_name= "lgbm opt")
study_lgbm.optimize(objective, n_trials= 50)
print('Best value: ', study_lgbm.best_value)
print('Best params: ', study_lgbm.best_params)

[I 2023-03-19 11:12:25,232] A new study created in memory with name: lgbm opt


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.436494 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2023-03-19 11:12:53,170] Trial 0 finished with value: 2.204701262654878 and parameters: {'num_leaves': 131, 'lambda_l1': 6.275322266205531, 'lambda_l2': 1.3606079157210431, 'bagging_fraction': 0.973099331546361, 'feature_fraction': 0.6416473130261418}. Best is trial 0 with value: 2.204701262654878.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.455461 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2023-03-19 11:13:26,691] Trial 1 finished with value: 2.1414760122763172 and parameters: {'num_leaves': 193, 'lambda_l1': 5.529143567670685, 'lambda_l2': 3.5353425438207093, 'bagging_fraction': 0.7938408257184608, 'feature_fraction': 0.7262754473464605}. Best is trial 1 with value: 2.1414760122763172.


...

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.447716 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2023-03-19 11:20:24,847] Trial 48 pruned. Trial was pruned at iteration 1.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.470748 seconds.
You can set `force_col_wise=true` to remove the overhead.


[I 2023-03-19 11:20:31,186] Trial 49 pruned. Trial was pruned at iteration 0.


Best value:  2.1414760122763172
Best params:  {'num_leaves': 193, 'lambda_l1': 5.529143567670685, 'lambda_l2': 3.5353425438207093, 'bagging_fraction': 0.7938408257184608, 'feature_fraction': 0.7262754473464605}


In [20]:
lgbm_params = study_lgbm.best_params
lgbm_model = lgb.LGBMRegressor(**lgbm_params)
lgbm_model.fit(x_train_set, y_train_set)

[LightGBM] [Warning] bagging_fraction is set=0.7938408257184608, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7938408257184608
[LightGBM] [Warning] lambda_l1 is set=5.529143567670685, reg_alpha=0.0 will be ignored. Current value: lambda_l1=5.529143567670685
[LightGBM] [Warning] feature_fraction is set=0.7262754473464605, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7262754473464605
[LightGBM] [Warning] lambda_l2 is set=3.5353425438207093, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.5353425438207093


LGBMRegressor(bagging_fraction=0.7938408257184608,
              feature_fraction=0.7262754473464605, lambda_l1=5.529143567670685,
              lambda_l2=3.5353425438207093, num_leaves=193)

In [21]:
test_pred = lgbm_model.predict(test[features])
test_pred_lgbm = test_pred
res = {
    'contest-tmp2m-14d__tmp2m': test_pred,
    'index': test_data["index"]
}
res = pd.DataFrame(res)
res.head()

,contest-tmp2m-14d__tmp2m,index
0,27.131038,375734
1,27.141886,375735
2,27.179241,375736
3,27.506303,375737
4,27.842126,375738


In [22]:
res.to_csv("/kaggle/working/lgbm.csv", index= False)

### Trying Catboost

In [23]:
def objective(trial):
    
    params = {
        'max_depth': trial.suggest_int('max_depth', 6, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 250, 1000, step=250),
        'max_bin': trial.suggest_int('max_bin', 100, 300),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 32, 512),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0001, 1.0, log = True),
        "early_stopping_rounds" : 10,
        'random_state': 42,
        'verbose': 0,
        "eval_metric" : 'RMSE',
        "task_type" : "GPU",
        "devices" : '0'
    }

    model = CatBoostRegressor(**params)

    model.fit(
        x_train_set, y_train_set,
        eval_set=(x_eval_set, y_eval_set),
        cat_features=['id'],
        verbose=False,
        plot=True
    )

    y_pred = model.predict(x_eval_set)
    score = mean_squared_error(y_eval_set, y_pred, squared= False)
    return score

In [24]:
study_cat = optuna.create_study(direction= 'minimize', study_name= "cat opt")
study_cat.optimize(objective, n_trials= 50)
print('Best value: ', study_cat.best_value)
print('Best params: ', study_cat.best_params)
cat_params = study_cat.best_params

[I 2023-03-19 11:21:21,720] A new study created in memory with name: cat opt


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

[I 2023-03-19 11:23:07,329] Trial 0 finished with value: 2.03321014142542 and parameters: {'max_depth': 10, 'learning_rate': 0.0588344504406283, 'n_estimators': 500, 'max_bin': 241, 'min_data_in_leaf': 410, 'l2_leaf_reg': 0.0006407520226101355}. Best is trial 0 with value: 2.03321014142542.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

[I 2023-03-19 11:23:20,166] Trial 1 finished with value: 2.4008815908467813 and parameters: {'max_depth': 7, 'learning_rate': 0.03841381973664089, 'n_estimators': 250, 'max_bin': 271, 'min_data_in_leaf': 343, 'l2_leaf_reg': 0.0013758653726468298}. Best is trial 0 with value: 2.03321014142542.


...

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

[I 2023-03-19 11:35:44,156] Trial 48 finished with value: 2.18062481277554 and parameters: {'max_depth': 10, 'learning_rate': 0.06333600125657081, 'n_estimators': 750, 'max_bin': 142, 'min_data_in_leaf': 294, 'l2_leaf_reg': 0.00011886770099299058}. Best is trial 0 with value: 2.03321014142542.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

[I 2023-03-19 11:35:55,131] Trial 49 finished with value: 2.211235387899279 and parameters: {'max_depth': 9, 'learning_rate': 0.0531110940868037, 'n_estimators': 250, 'max_bin': 203, 'min_data_in_leaf': 357, 'l2_leaf_reg': 0.00017258504305519746}. Best is trial 0 with value: 2.03321014142542.


Best value:  2.03321014142542
Best params:  {'max_depth': 10, 'learning_rate': 0.0588344504406283, 'n_estimators': 500, 'max_bin': 241, 'min_data_in_leaf': 410, 'l2_leaf_reg': 0.0006407520226101355}


In [25]:
cat_params = study_cat.best_params
cat_model = CatBoostRegressor(**cat_params)
cat_model.fit(x_train_set, y_train_set, cat_features= ['id'])

0:	learn: 9.0560276	total: 1.36s	remaining: 11m 19s
1:	learn: 8.5867204	total: 2.71s	remaining: 11m 14s
2:	learn: 8.1370758	total: 4.06s	remaining: 11m 13s
3:	learn: 7.7264539	total: 5.72s	remaining: 11m 49s
4:	learn: 7.3340998	total: 7.09s	remaining: 11m 42s
5:	learn: 6.9660709	total: 8.4s	remaining: 11m 31s
6:	learn: 6.6173530	total: 9.77s	remaining: 11m 28s
7:	learn: 6.2865473	total: 11.1s	remaining: 11m 25s
8:	learn: 5.9781862	total: 12.5s	remaining: 11m 21s
9:	learn: 5.6882360	total: 13.8s	remaining: 11m 18s
10:	learn: 5.4167397	total: 15.2s	remaining: 11m 16s
11:	learn: 5.1616498	total: 16.9s	remaining: 11m 25s
12:	learn: 4.9192708	total: 18.2s	remaining: 11m 21s
13:	learn: 4.6924735	total: 19.5s	remaining: 11m 17s
14:	learn: 4.4761897	total: 20.9s	remaining: 11m 15s
15:	learn: 4.2752268	total: 22.2s	remaining: 11m 11s
16:	learn: 4.0858003	total: 23.5s	remaining: 11m 7s
17:	learn: 3.9058054	total: 24.8s	remaining: 11m 3s
18:	learn: 3.7365366	total: 26.1s	remaining: 10m 59s
19:	le

In [26]:
test_pred = cat_model.predict(test[features])
test_pred_cat = test_pred
res = {
    'contest-tmp2m-14d__tmp2m': test_pred,
    'index': test_data["index"]
}
res = pd.DataFrame(res)
res.head()

,contest-tmp2m-14d__tmp2m,index
0,22.904485,375734
1,22.525968,375735
2,22.675056,375736
3,22.295956,375737
4,22.488810,375738


In [27]:
res.to_csv("/kaggle/working/cat.csv", index= False)

### XGBoost with optuna

In [28]:
# def objective(trial):
#     params = {
#             'objective': "reg:squarederror",
#             'max_depth': trial.suggest_int('max_depth', 1, 9),
#             'learning_rate': trial.suggest_float('learning_rate', 0.01, 1.0),
#             'n_estimators': trial.suggest_int('n_estimators', 50, 500),
#             'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
#             'gamma': trial.suggest_float('gamma', 1e-8, 1.0),
#             'subsample': trial.suggest_float('subsample', 0.01, 1.0),
#             'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
#             'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0),
#             'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0),
#             'eval_metric': 'mlogloss',
#             'use_label_encoder': False
#         }
#     eval_set = [(x_eval_set, y_eval_set)]
#     xg_model = XGBRegressor(**params)
#     xg_model.fit(x_train_set, y_train_set, eval_set= eval_set, early_stopping_rounds= 10)
#     y_pred = xg_model.predict(x_eval_set)
#     score = mean_squared_error(y_eval_set, y_pred, squared= False)
#     return score

In [29]:
# study_xg = optuna.create_study(direction= 'minimize', study_name= "xg opt")
# study_xg.optimize(objective, n_trials= 50)
# print('Best value: ', study_xg.best_value)
# print('Best params: ', study_xg.best_params)

In [30]:
# xg_params = study_xg.best_params
# xg_model = XGBRegressor(**xg_params)
# # xg_model = XGBRegressor(objective="reg:squarederror", eval_metric='rmse', 
# #                              random_state=42)
# xg_model.fit(x_train_set, y_train_set)
# y_pred = xg_model.predict(x_eval_set)
# score = mean_squared_error(y_eval_set, y_pred, squared= False)

In [31]:
# test_pred = xg_model.predict(test[features])
# test_pred_xg = test_pred
# res = {
#     'contest-tmp2m-14d__tmp2m': test_pred,
#     'index': test_data["index"]
# }
# res = pd.DataFrame(res)
# res.head()

In [32]:
# res.to_csv("/kaggle/working/xg.csv", index= False)

### Ensemble

In [33]:
def ensembling(lgb_ratio, cat_ratio):
    ensemble_preds = test_pred_lgbm * lgb_ratio + test_pred_cat * (1 - lgb_ratio)
    return ensemble_preds

In [34]:
def export_ensemble(ratio_list, prev):
    for i in ratio_list:
        test_pred = ensembling(i[0], i[1])
        pr = prev
        pr["contest-tmp2m-14d__tmp2m"] = pr["contest-tmp2m-14d__tmp2m"] + test_pred
        res = pr[["contest-tmp2m-14d__tmp2m", "index"]]
        res.to_csv('submission.csv', index = False)

In [35]:
ratios = [
    [0.4, 0.6],
]
export_ensemble(ratios, sub)